In [141]:
import numpy as np
import seaborn as sns
from keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
import pandas as pd
import os
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.models import Model, Sequential
from keras.optimizers import Adam
import tensorflow as tf
import cv2
from sklearn.model_selection import train_test_split
import argparse
import locale



test_data = pd.read_csv('test.csv') 
train_data = pd.read_csv('train.csv') 

# default image size: 48*48 pixels
pic_size = 48

# input path for the images
base_path = "../IDS2021_HW09/"


#We select the necessary metadata and convert it into the necessary form
needed_data = train_data[['Id', 'Pawpularity']]
size = len(needed_data)
needed_data['Pawpularity'] = needed_data['Pawpularity'].astype(str)
i=0
for expression in os.listdir(base_path + "files/train/"):
    needed_data['Id'].iloc[i] = expression
    i+=1



#We load all the images into an array
images = []
for filename in needed_data['Id']:
    image = cv2.imread(base_path + "files/train/" + filename)
    image = cv2.resize(image, (64, 64))
    images.append(image)

    
image_array = np.array(images)

    
#funtion to make the neural network
def make_cnn_model(image_width, image_height, image_depth, filters=(16, 32, 64), regress=False):
    Shape = (image_height, image_width, image_depth)
    inputs = Input(shape=Shape)
    for (i, f) in enumerate(filters):
        if i == 0:
            x = inputs
        x = Conv2D(f, (3, 3), padding="same")(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=-1)(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    x = Dense(20)(x)
    x = Activation("relu")(x)
    x = BatchNormalization(axis=-1)(x)
    x = Dropout(0.5)(x)
    x = Dense(8)(x)
    x = Activation("relu")(x)
    if regress:
        x = Dense(1, activation="linear")(x)
    model = Model(inputs, x)
    return model

image_array = image_array /255.0
split = train_test_split(train_data['Pawpularity'], image_array, test_size=0.25, random_state=40)
(train, test, trainImages, testImages) = split


#We convert the pawpularity to be between [0,1]
maxPrice = train.max()
trainY = train / maxPrice
testY = test / maxPrice
#We make and train the model
model2 = make_cnn_model(64, 64, 3, regress=True)
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model2.compile(loss="mean_absolute_percentage_error", optimizer=opt)

print("[INFO] training model...")
model2.fit(x=trainImages, y=trainY, 
    validation_data=(testImages, testY),
    epochs=200, batch_size=8)

<ipython-input-141-25b54ff3c7e2>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  needed_data['Pawpularity'] = needed_data['Pawpularity'].astype(str)
C:\Users\uzingel\Anaconda3\lib\site-packages\pandas\core\indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


[INFO] training model...
Epoch 1/200
930/930 [==============================] - 44s 47ms/step - loss: 129.2804 - val_loss: 72.5946
Epoch 2/200
930/930 [==============================] - 47s 51ms/step - loss: 71.8589 - val_loss: 65.4726
Epoch 3/200
930/930 [==============================] - 45s 49ms/step - loss: 62.4522 - val_loss: 59.2160
Epoch 4/200
930/930 [==============================] - 44s 48ms/step - loss: 58.9498 - val_loss: 57.4920
Epoch 5/200
930/930 [==============================] - 45s 49ms/step - loss: 59.0065 - val_loss: 57.5426
Epoch 6/200
930/930 [==============================] - 44s 48ms/step - loss: 58.6963 - val_loss: 58.2752
Epoch 7/200
930/930 [==============================] - 46s 50ms/step - loss: 57.8622 - val_loss: 57.9392
Epoch 8/200
930/930 [==============================] - 48s 51ms/step - loss: 57.6338 - val_loss: 57.7466
Epoch 9/200
930/930 [==============================] - 44s 47ms/step - loss: 57.5846 - val_loss: 57.5907
Epoch 10/200
930/930 [=======

930/930 [==============================] - 43s 46ms/step - loss: 57.1392 - val_loss: 57.1379
Epoch 79/200
930/930 [==============================] - 41s 44ms/step - loss: 57.0012 - val_loss: 57.2450
Epoch 80/200
930/930 [==============================] - 41s 44ms/step - loss: 57.1725 - val_loss: 57.1816
Epoch 81/200
930/930 [==============================] - 41s 44ms/step - loss: 57.1508 - val_loss: 57.2091
Epoch 82/200
930/930 [==============================] - 40s 43ms/step - loss: 57.1376 - val_loss: 57.1548
Epoch 83/200
930/930 [==============================] - 41s 45ms/step - loss: 57.1134 - val_loss: 57.1823
Epoch 84/200
930/930 [==============================] - 42s 45ms/step - loss: 57.1060 - val_loss: 57.1350
Epoch 85/200
930/930 [==============================] - 42s 45ms/step - loss: 57.0655 - val_loss: 57.1374
Epoch 86/200
930/930 [==============================] - 40s 43ms/step - loss: 57.1364 - val_loss: 57.1371
Epoch 87/200
930/930 [==============================] - 43s

Epoch 155/200
930/930 [==============================] - 32s 35ms/step - loss: 56.0122 - val_loss: 58.2337
Epoch 156/200
930/930 [==============================] - 32s 35ms/step - loss: 55.6765 - val_loss: 58.4318
Epoch 157/200
930/930 [==============================] - 34s 37ms/step - loss: 55.7834 - val_loss: 58.5795
Epoch 158/200
930/930 [==============================] - 33s 35ms/step - loss: 55.7229 - val_loss: 58.5344
Epoch 159/200
930/930 [==============================] - 34s 37ms/step - loss: 55.2508 - val_loss: 58.6045
Epoch 160/200
930/930 [==============================] - 33s 36ms/step - loss: 55.5752 - val_loss: 58.5787
Epoch 161/200
930/930 [==============================] - 35s 38ms/step - loss: 55.2328 - val_loss: 58.8571
Epoch 162/200
930/930 [==============================] - 33s 36ms/step - loss: 55.7288 - val_loss: 58.1013
Epoch 163/200
930/930 [==============================] - 32s 35ms/step - loss: 55.0180 - val_loss: 59.0210
Epoch 164/200
930/930 [==============

In [ ]:
#Predict the values of images
preds = model2.predict(testImages)

#Calculate the differences between results and the real values
diff = preds.flatten() - testY
percentDiff = (diff / testY) * 100
absPercentDiff = np.abs(percentDiff)

#Calculate the mean of our prediction accuracy
mean = np.mean(absPercentDiff)
print(mean)

In [92]:
#Predict the values for the test data

needed_data_test = test_data
j = 0
for expression in os.listdir(base_path + "test/"):
    needed_data_test['Id'].iloc[j] = expression
    j+=1


images2 = []
for filename in needed_data_test['Id']:
    image2 = cv2.imread(base_path + "test/" + filename)
    image2 = cv2.resize(image2, (64, 64))
    images2.append(image2)

    
image_array2 = np.array(images2)
image_array2 = image_array2 /255.0

#Results
predict2 = predict2 * 100
predict2 = model2.predict(image_array2)

In [ ]:

#Here we work with the metadata

from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


#We split the data into validation and training
validation = train_data.iloc[0:40]
training = train_data.iloc[40:]

#We assign different types of data to variables
y_train = training['Pawpularity']
X_train = training.drop(columns=['Pawpularity', 'Id'])
X_test = validation.drop(columns=['Id', 'Pawpularity'])
y_test = validation['Pawpularity']

#Treeclassifier
model3 = tree.DecisionTreeClassifier(random_state=0, criterion='gini')
model3.fit(X_train, y_train)
results = model3.predict(X_test)
accuracy1 = accuracy_score(y_test, model3.predict(X_test))
#print(results)

#KN
model4 = KNeighborsClassifier(n_neighbors = 11)
model4.fit(X_train, y_train)
results2 = model4.predict(X_test)
accuracy2 = accuracy_score(y_test, model4.predict(X_test))
#print(results2)

#KN with manhattan metric
model5 = KNeighborsClassifier(n_neighbors = 11, metric="manhattan")
model5.fit(X_train, y_train)
results3 = model5.predict(X_test)
accuracy3 = accuracy_score(y_test, model5.predict(X_test))
#print(results3)

#print(y_test)


#As we can see, the accuracy is very low with just the metadata
print(accuracy1)
print(accuracy2)
print(accuracy3)

In [ ]:
results_df = pd.DataFrame(columns=['model', 'accuracy'])
results_df = results_df.append({'model': 'DecisionTreeClassifier', 'accuracy': accuracy1 }, ignore_index=True)
results_df = results_df.append({'model': 'KNeighborsClassifier', 'accuracy': accuracy2 }, ignore_index=True)
results_df = results_df.append({'model': 'KNeighborsClassifier(manhattan)', 'accuracy': accuracy3 }, ignore_index=True)
results_df